# BlueMap Screenshot Testing

Test rendering Minecraft schematics using BlueMap CLI.

Based on:
- https://bluemap.bluecolored.de/community/python-screenshots.html
- https://bluemap.bluecolored.de/wiki/getting-started/Installation.html

In [ ]:
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))
sys.path.insert(0, str(Path.cwd().parent / 'scripts'))

from bluemap_screenshots import BlueMapRenderer
from minecraft_voxel_flow.loaders.schematic_loader import SchematicLoader
from minecraft_voxel_flow.visualization.viewers import SchematicViewer

import matplotlib.pyplot as plt
from IPython.display import Image, display
import numpy as np

## 1. Initialize BlueMap Renderer

In [ ]:
renderer = BlueMapRenderer()

# Check if everything is ready
if renderer.check_requirements():
    print("\n✅ BlueMap is ready to use!")
else:
    print("\n❌ BlueMap setup incomplete. Check the output above.")

## 2. Find Available Schematics

In [ ]:
schematics_dir = Path("../data/schematics")

# Find all schematic files
schematic_files = (
    list(schematics_dir.glob("*.schematic")) +
    list(schematics_dir.glob("*.schem")) +
    list(schematics_dir.glob("*.litematic"))
)

print(f"Found {len(schematic_files)} schematic files")
print(f"\nFirst 10 files:")
for f in schematic_files[:10]:
    print(f"  - {f.name}")

## 3. Test with a Single Schematic

Let's first look at what the schematic contains using our existing loader:

In [ ]:
# Pick a schematic to test
test_schematic = schematic_files[0]
print(f"Testing with: {test_schematic.name}")

# Load it to see what it looks like
loader = SchematicLoader()
voxels, palette = loader.load(str(test_schematic))

print(f"\nSchematic info:")
print(f"  Shape: {voxels.shape}")
print(f"  Unique blocks: {len(palette)}")
print(f"  Non-air blocks: {np.sum(voxels > 0)}")

# Get statistics
stats = loader.get_statistics(voxels, palette)
print(f"\nTop 10 blocks:")
for block, count in stats['top_blocks'][:10]:
    print(f"  - {block}: {count}")

## 4. View with Matplotlib (Quick Preview)

In [ ]:
viewer = SchematicViewer()

# Show 2D slices
viewer.view_slices(voxels, palette=palette, title=test_schematic.stem)
plt.tight_layout()
plt.show()

## 5. Test BlueMap Rendering

Now let's try rendering with BlueMap for a high-quality 3D visualization.

In [ ]:
output_dir = Path("../data/bluemap_test")
output_dir.mkdir(exist_ok=True)

output_path = output_dir / f"{test_schematic.stem}.png"

print(f"Rendering {test_schematic.name} with BlueMap...")
print(f"Output: {output_path}")

success = renderer.render_schematic(
    str(test_schematic),
    str(output_path),
    camera_distance=100,
    camera_angle=45,
    camera_rotation=45
)

if success and output_path.exists():
    print("\n✅ Render successful!")
    display(Image(filename=str(output_path)))
else:
    print("\n⚠️  Render incomplete or in progress...")
    print("Note: BlueMap CLI may require additional setup for direct schematic rendering.")

## 6. Alternative: Generate Matplotlib Previews

While we work on the BlueMap integration, let's use the matplotlib renderer to create previews:

In [ ]:
# Save matplotlib preview
preview_path = output_dir / f"{test_schematic.stem}_preview.png"

viewer.save_render(
    voxels,
    str(preview_path),
    palette=palette,
    view_type='slices',
    figsize=(15, 10),
    title=test_schematic.stem
)

print(f"✅ Matplotlib preview saved to: {preview_path}")
display(Image(filename=str(preview_path)))

## 7. Batch Preview Generation

Generate previews for multiple schematics:

In [ ]:
from tqdm.notebook import tqdm

# Generate previews for first 10 schematics
num_to_render = 10
preview_dir = Path("../data/previews")
preview_dir.mkdir(exist_ok=True)

print(f"Generating previews for {num_to_render} schematics...")

for schematic_path in tqdm(schematic_files[:num_to_render]):
    try:
        # Load
        voxels, palette = loader.load(str(schematic_path))
        
        # Save preview
        output_path = preview_dir / f"{schematic_path.stem}.png"
        viewer.save_render(
            voxels,
            str(output_path),
            palette=palette,
            view_type='slices',
            figsize=(12, 8),
            title=schematic_path.stem
        )
        
    except Exception as e:
        print(f"\n❌ Failed to render {schematic_path.name}: {e}")

print(f"\n✅ Previews saved to: {preview_dir}")

## 8. Display Gallery of Generated Previews

In [ ]:
# Show grid of previews
preview_files = list(preview_dir.glob("*.png"))[:12]

if preview_files:
    n_cols = 3
    n_rows = (len(preview_files) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 6*n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes
    
    for idx, preview_file in enumerate(preview_files):
        if idx < len(axes):
            img = plt.imread(preview_file)
            axes[idx].imshow(img)
            axes[idx].set_title(preview_file.stem, fontsize=10)
            axes[idx].axis('off')
    
    # Hide unused subplots
    for idx in range(len(preview_files), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("No preview files found. Run the batch generation cell first.")

## 9. Compare Different View Types

In [ ]:
# Pick a schematic
test_idx = 5
if test_idx < len(schematic_files):
    test_file = schematic_files[test_idx]
    voxels, palette = loader.load(str(test_file))
    
    print(f"Comparing views for: {test_file.name}")
    print(f"Shape: {voxels.shape}")
    
    # 2D Slices
    fig = viewer.view_slices(voxels, palette=palette, title=f"{test_file.stem} - 2D Slices")
    plt.show()
    
    # 3D Scatter (if not too large)
    if np.sum(voxels > 0) < 10000:  # Only if reasonable number of blocks
        fig = viewer.view_3d(voxels, palette=palette, title=f"{test_file.stem} - 3D View")
        fig.show()
    else:
        print(f"\nSkipping 3D view - too many blocks ({np.sum(voxels > 0)})")

## 10. Batch Process All Schematics (Optional)

⚠️ **Warning**: This will process all schematics. Consider running with smaller batches first.

In [ ]:
# Uncomment to run batch processing
# from generate_previews import generate_previews

# generate_previews(
#     schematics_dir="../data/schematics",
#     output_dir="../data/previews_all",
#     max_schematics=100,  # Start with 100
#     view_type="slices",
#     figsize=(15, 10)
# )

print("Uncomment the code above to run batch processing.")

## Summary

This notebook demonstrates:
1. ✅ Loading schematics with SchematicLoader
2. ✅ Viewing with matplotlib (2D slices and 3D scatter)
3. ✅ Batch generating preview images
4. ⏳ BlueMap integration (requires additional setup for schematic conversion)

For production use, the matplotlib-based previews are fast and reliable. BlueMap would provide higher quality 3D renders but requires converting schematics to Minecraft worlds first.